In [11]:
# Install dependencies
!pip install -U guidance guardrails-ai llama-cpp-python=="0.2.26" huggingface_hub

^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
TRANSCRIPT = """This is astronaut Neil Armstrong, command pilot for the Apollo 11 moon landing mission.
What is the purpose of the Apollo 11 mission? Apollo 11 is man's first attempt to demonstrate the ability to go to the moon, to land there, and to return to Earth. ...and the module pilot.
Well, Apollo was designed to be a three-man job, and the third which I perform is, I think, as important, no more so, no less so, than the other two positions.
I think I'd be a fool if I said that I had the best seat of the three. On the other hand, I can say with complete candor and with complete honesty that I'm very happy.
I'm very happy to have the seat which I have, and to be doing the job that I intend to do. Roger, Eagle. Roger, how does it look? Eagle has wings. Roger.
Would you describe what will be happening just before the lunar module touches down on the moon? We will continue burning the engine until an altitude of only about five feet, at which the probes will ignite.
We see a light on our panel. We see it illuminate. We cut the engine and thaw the last several feet to touch down. ...forward, drifting to the right a little. Thirty seconds. ...contact light.
Okay, engine stop. We copy you down, Eagle. ...trangulity base here. The Eagle has landed. About exploring the moon's surface, astronaut Aldrin had this to say.
Following the EVA preparation period, we will go through the planned two-man excursion on the surface with Neil exiting the spacecraft first.
I'm going to step off the landing. That's one small step for man, one giant leap for mankind."""

In [ ]:
# import os
# os.environ["OPENAI_API_KEY"] = ...

In [ ]:
# Guardrails Example
import os
import json
from typing import List
from pydantic import BaseModel, Field
import guardrails as gd
from guardrails.validators import ValidChoices
import openai

# Make sure you set your API key for OpenAI to run this example
# Skip to the local model one if you don't have an API key or create one!
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", None)
if OPENAI_API_KEY is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

# Define the Pydantic model for LLM Output Validation
class TranscriptSection(BaseModel):
    speaker: str = Field(description="The speaker that is speaking the transcript section.", validators=[ValidChoices(choices=["Speaker 1", "Speaker 2"])])
    sentences: str= Field(description="The sentences spoken by the speaker.")

class TranscriptWithSpeakers(BaseModel):
    transcript: List[TranscriptSection] = Field(description="The transcript with speakers.")


# Define the Prompt for the task.
# This is very basic and could easily be improved!
prompt = """
Given the following transcript, please extract a dictionary that contains each each transcript section and its speaker.
Focus on identifying different parties in the conversation and assume there are two speakers. Use the speaker names "Speaker 1" and "Speaker 2"

${transcript}

${gr.complete_json_suffix_v2}
"""

# Create a Guard object from the Pydantic model to prompt the LLM
guard = gd.Guard.from_pydantic(output_class=TranscriptWithSpeakers, prompt=prompt)

# Prompt the Model
# Guardrails will reask the LLM if the output is not valid
res = guard(
    openai.chat.completions.create,
    prompt_params={"transcript": TRANSCRIPT},
    model="gpt-3.5-turbo-0125",
    max_tokens=4096,
    num_reasks=2,
    temperature=0.0,
)

# Print the VALIDATED output from the LLM
print(json.dumps(res.validated_output, indent=2))

{
  "transcript": [
    {
      "speaker": "Speaker 1",
      "sentences": "This is astronaut Neil Armstrong, command pilot for the Apollo 11 moon landing mission."
    },
    {
      "speaker": "Speaker 2",
      "sentences": "What is the purpose of the Apollo 11 mission? Apollo 11 is man's first attempt to demonstrate the ability to go to the moon, to land there, and to return to Earth. ...and the module pilot."
    },
    {
      "speaker": "Speaker 1",
      "sentences": "Well, Apollo was designed to be a three-man job, and the third which I perform is, I think, as important, no more so, no less so, than the other two positions."
    },
    {
      "speaker": "Speaker 2",
      "sentences": "I think I'd be a fool if I said that I had the best seat of the three. On the other hand, I can say with complete candor and with complete honesty that I'm very happy."
    },
    {
      "speaker": "Speaker 1",
      "sentences": "I'm very happy to have the seat which I have, and to be doing t

In [ ]:
# Download the mistral model for local use before running the example below (!)
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf to /home/daniel/.cache/huggingface/hub/tmphtnd568f
mistral-7b-instruct-v0.2.Q4_K_M.gguf: 100%|█| 4.37G/4.37G [06:35<00:00, 11.0MB/s
./mistral-7b-instruct-v0.2.Q4_K_M.gguf


In [18]:
# Guidance Example
from guidance import models, gen, select
from guidance import user, system, assistant


prompt = f"""
Given the following transcript, please extract a dictionary that contains each each transcript section and its speaker.
Focus on identifying different parties in the conversation and assume there are two speakers, one being the interviewer and narrator and the other one answering questions.
Use the speaker names "Speaker 1" and "Speaker 2".

Use the format:
<Part of the transcript> -> Speaker 1

<Part of the transcript> -> Speaker 1

<Part of the transcript> -> Speaker 2

... (repeat for each sentence)

Transcript:
"{TRANSCRIPT}"\n\n"""

llm = models.LlamaCppChat("mistral-7b-instruct-v0.2.Q4_K_M.gguf", n_ctx=4096)

with system():
    llm += "You are a helpful assistant.\n"

with user():
    llm += prompt

with assistant():
    total_length = len(TRANSCRIPT)
    processed_length = 0
    transcript_sections = []
    while processed_length < total_length:
        llm += "\"" +  gen(stop=["\""], name="transcript_section") + "\" -> " + gen(regex=r"Speaker (1|2)", name="speaker", suffix="\n\n")
        transcript_sections.append({"speaker": llm["speaker"], "transcript_section": llm["transcript_section"]})
        processed_length += len(llm["transcript_section"])
    


